In [17]:
from bins import Bins
from utils import calcAllChi2, calcOneChi2, makeHists
from ROOT import TFile, TH1

bins = Bins.readFrom("ranges.yml")
histsData = makeHists("apr12_diele_088_090_ag123ag_2500A_accepted_np_2.dat", "_data", bins)

#outfile = TFIle("out.root","RECREATE")
allHistsMC = []

#calcAllChi2(histsMC, histsData)

iter 9
Before processing events 23:13:26


  0%|          | 0/38546 [00:00<?, ?it/s]

After processing events 23:13:26


Warning in <TROOT::Append>: Replacing existing TH1: hmass_data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hz_data (Potential memory leak).


In [18]:
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

In [19]:
TH1.SetDefaultSumw2

N_PARAMS = 3

grid_x = torch.linspace(-1,1,101)

grid_x1, grid_x2, grid_x3 = torch.meshgrid(grid_x, grid_x, grid_x, indexing="ij")

xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten(), grid_x3.flatten()]).transpose(-2,-1)

In [20]:
lb = -1
ub = 1

bounds = torch.tensor([[lb]*N_PARAMS, [ub]*N_PARAMS], dtype=torch.float)
bounds

tensor([[-1., -1., -1.],
        [ 1.,  1.,  1.]])

In [21]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [22]:
num_queries = 3
num_repeats = 1
num_samples = 5

In [23]:
def lambdas(x):
    return x[0], x[1], x[2]

def cost(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

            result = 0

            if pow(1-lambda_phi, 2) - pow(lambda_theta-lambda_phi, 2) < 4*pow(lambda_theta_phi, 2):
                result = 1
            if 1 + lambda_theta + 2*lambda_phi < 0:
                result = 1

            yield torch.tensor([result])
    return torch.stack([a for a in generator(xx)])
    
def objective(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

            histsMC = makeHists("medium_isotropic_eff_ag1230ag_np_9deg.dat", "_MC", bins, lambda_theta, lambda_phi, lambda_theta_phi)
            chi2, ndf = calcOneChi2(histsMC[0][0], histsData[0][0])
            allHistsMC.append(histsMC[0][0])
            if not chi2 or not ndf:
                return torch.tensor([0])
            yield torch.tensor([1./(chi2 / ndf)])
    return torch.stack([a for a in generator(xx)])

In [24]:
strategy = "cei"  # "cei" or "ei"
strategy = strategy.upper()

# -2 is the default value when no feasible has been found
default_value = -2
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(num_samples, 3)
    train_utility = objective(train_x)
    train_cost = cost(train_x)

    for i in tqdm(range(num_queries)):
        print("query", i)

        feasible_flag = (train_cost <= 0).any()

        if feasible_flag:
            feasible_incumbents[trial, i] = train_utility[train_cost <= 0].max()

        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        cost_model, cost_likelihood = fit_gp_model(train_x, train_cost.squeeze(-1))

        if feasible_flag:
            best_f = train_utility[train_cost <= 0].max()
        else:
            best_f = torch.tensor(default_value)

        if strategy == "EI":
            policy = botorch.acquisition.analytic.ExpectedImprovement(
                model=utility_model,
                best_f=train_utility.max(),
            )

        if strategy == "CEI":
            policy = botorch.acquisition.analytic.ConstrainedExpectedImprovement(
                model=botorch.models.model_list_gp_regression.ModelListGP(
                    utility_model, cost_model
                ),
                best_f=best_f,
                objective_index=0,
                constraints={1: [-1,1]}
            )
        
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

        next_utility = objective(next_x)
        next_cost = cost(next_x)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])
        train_cost = torch.cat([train_cost, next_cost])

trial 0
iter 10
Before processing events 23:14:01


  0%|          | 0/1385092 [00:00<?, ?it/s]

After processing events 23:14:32
iter 11
Before processing events 23:15:07


  0%|          | 0/1385092 [00:00<?, ?it/s]

After processing events 23:15:38
iter 12
Before processing events 23:16:13


...

  0%|          | 0/1385092 [00:00<?, ?it/s]

After processing events 23:21:12
query 2
iter 17
Before processing events 23:21:48


  0%|          | 0/1385092 [00:00<?, ?it/s]

After processing events 23:22:20


Warning in <TROOT::Append>: Replacing existing TH1: hmass_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hz_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hmass_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hz_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hmass_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hz_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hmass_MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hz_MC (Potential memory leak).


In [25]:
torch.save(feasible_incumbents, f"./{strategy}.pth")
fout = TFile("out.root", "RECREATE")
fout.cd()
for hist in allHistsMC:
    print ("Writing hist: ", hist.GetName())
    hist.Write()
for j, hists in enumerate(histsData):
    for k, hist in enumerate(hists):
            hist.Write()
            print ("Writing hist: ", j, k, hist)
fout.Close()

Writing hist:  hist_0to150_m06m02_MC_iter10
Writing hist:  hist_0to150_m06m02_MC_iter11
Writing hist:  hist_0to150_m06m02_MC_iter12
Writing hist:  hist_0to150_m06m02_MC_iter13
Writing hist:  hist_0to150_m06m02_MC_iter14
Writing hist:  hist_0to150_m06m02_MC_iter15
Writing hist:  hist_0to150_m06m02_MC_iter16
Writing hist:  hist_0to150_m06m02_MC_iter17
Writing hist:  0 0 Name: hist_0to150_m06m02_data_iter9 Title: hist_0to150_m06m02_data_iter9 NbinsX: 20
Writing hist:  0 1 Name: hist_0to150_m02p01_data_iter9 Title: hist_0to150_m02p01_data_iter9 NbinsX: 20
Writing hist:  0 2 Name: hist_0to150_p01p08_data_iter9 Title: hist_0to150_p01p08_data_iter9 NbinsX: 20
Writing hist:  0 3 Name: hist_150to300_m06m02_data_iter9 Title: hist_150to300_m06m02_data_iter9 NbinsX: 20
Writing hist:  0 4 Name: hist_150to300_m02p01_data_iter9 Title: hist_150to300_m02p01_data_iter9 NbinsX: 20
Writing hist:  0 5 Name: hist_150to300_p01p08_data_iter9 Title: hist_150to300_p01p08_data_iter9 NbinsX: 20
Writing hist:  0 6

In [26]:
t = torch.tensor( [ [-1.0000, -0.4678,  0.5433]])
print(t)
print(t.shape)
cost(t)

tensor([[-1.0000, -0.4678,  0.5433]])
torch.Size([1, 3])


tensor([[1]])

In [27]:
with torch.no_grad():
    predictive_distribution = cost_likelihood(cost_model(xs))
    predictive_mean = predictive_distribution.mean
    predictive_lower, predictive_upper = predictive_distribution.confidence_region()


In [35]:
from ipywidgets import interact, Layout, IntSlider
import numpy as np

def oneplot(ax, tensor, index, cmap):
        tensor_3d = torch.reshape(tensor, (101,101,101))
        pos = ax.imshow(tensor_3d[index], cmap=cmap, interpolation="nearest", origin="lower", 
        vmin=tensor.min(), vmax=tensor.max(), extent=[-1,1,-1,1])
        plt.colorbar(pos)

def f(x):
        cmap = "gist_rainbow"
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))
        oneplot(ax[0][0], predictive_mean, x, cmap)
        oneplot(ax[0][1], predictive_lower, x, cmap)
        oneplot(ax[1][1], predictive_upper, x, cmap)
        oneplot(ax[1][0], predictive_upper-predictive_lower, x, cmap)

interact(f, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))


interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.f(x)>

In [29]:
with torch.no_grad():
    predictive_distribution_cost = cost_likelihood(cost_model(xs))
    predictive_mean_cost = predictive_distribution.mean
    predictive_lower_cost, predictive_upper_cost = predictive_distribution.confidence_region()


def g(x):
        cmap = "cool"
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))
        oneplot(ax[0][0], predictive_mean_cost, x, cmap)
        oneplot(ax[0][1], predictive_lower_cost, x, cmap)
        oneplot(ax[1][1], predictive_upper_cost, x, cmap)
        oneplot(ax[1][0], predictive_upper_cost-predictive_lower_cost, x, cmap)


interact(g, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))

interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.g(x)>

In [30]:
a = torch.tensor([1,2,3])
b = torch.Tensor([4,5,6])
torch.stack([a,b])

tensor([[1., 2., 3.],
        [4., 5., 6.]])